In [4]:
import cv2
import face_recognition
import os
import math
import numpy
from datetime import datetime

videoCapture = cv2.VideoCapture(0)

def markAttendance(regNumber, name):
    f = open("Attendance.csv", "r+")
    data = f.readlines()
    regNumberList = []
    for line in data:
        entry = line.split(',')
        regNumberList.append(entry[0])
    if regNumber not in regNumberList:
        time = datetime.now()
        timeFormat = time.strftime('%H:%M:%S')
        f.writelines(f'\n{regNumber},{name},{timeFormat}')
        print(f"Pengguna {name} dengan nomor registrasi {regNumber} berhasil ditambahkan.")

def getAccuracy(faceDistance, faceMatchThreshold = 0.6):
    if faceDistance > faceMatchThreshold:
        range = (1.0 - faceMatchThreshold)
        linearValue = (1.0 - faceDistance) / (range * 2.0)
        return linearValue
    else:
        range = faceMatchThreshold
        linearValue = 1.0 - (faceDistance / (range * 2.0))
        return linearValue + ((1.0 - linearValue) * math.pow((linearValue - 0.5) * 2, 0.2))

def addNewUser():
    ret, frame = videoCapture.read()

    imageName = input("Masukkan nama file gambar (format: Nama.NomorRegistrasi.jpg): ")
    cv2.imwrite(f"imageData/{imageName}", frame)

    reloadDataset()

def reloadDataset():
    global allPaths, allNames, allRegNumbers, allEncodings

    allPaths = os.listdir("imageData")
    allNames = []
    allRegNumbers = []
    allEncodings = []
    for index in range(len(allPaths)):
        allNames.append(allPaths[index].split(".")[0])
        allRegNumbers.append(allPaths[index].split(".")[1])
        image = face_recognition.load_image_file("imageData/" + allPaths[index])
        temp = face_recognition.face_encodings(image)[0]
        allEncodings.append(temp)

reloadDataset()

while True:
    ret, frame = videoCapture.read()

    frame = cv2.resize(frame, (0, 0), fx=2, fy=1.6)

    resizedFrame = cv2.resize(frame, (0, 0), fx=0.2, fy=0.2)

    requiredFrame = cv2.cvtColor(resizedFrame, cv2.COLOR_BGR2RGB)

    faceLocation = face_recognition.face_locations(requiredFrame)

    faceEncoding = face_recognition.face_encodings(requiredFrame, faceLocation)

    faceNames = []
    for encoding in faceEncoding: 
        ismatched = face_recognition.compare_faces(allEncodings, encoding)
    matchedName = "Unknown"

    faceDistance = face_recognition.face_distance(allEncodings, encoding)

    if len(faceDistance) > 1:
        if faceDistance[0] > faceDistance[1]:
            minimumFaceDistance = faceDistance[1]
        else:
            minimumFaceDistance = faceDistance[0]
    else:
        minimumFaceDistance = faceDistance[0]

    accuracy = getAccuracy(minimumFaceDistance)*100

    bestMatchIndex = numpy.argmin(faceDistance)

    if ismatched[bestMatchIndex] and accuracy > 80:
        matchedName = allNames[bestMatchIndex]
        markAttendance(allRegNumbers[bestMatchIndex], matchedName)

    faceNames.append(matchedName)


    for (top, right, bottom, left), name in zip(faceLocation, faceNames):
        top *= 5
        right *= 5
        bottom *= 5
        left *= 5

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)
        cv2.putText(frame, name, (left + 6, bottom - 10), cv2.FONT_HERSHEY_DUPLEX, 1.5, (0, 255, 0), 2)
        if(accuracy > 80):
            cv2.putText(frame, "%.2f"%accuracy + "%", (left + 6, bottom + 30), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 255, 0), 2)
    
    cv2.putText(frame, "Tekan 'A' untuk tambahkan pengguna baru", (20, 50), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 255, 0), 2)
    cv2.putText(frame, "Tekan 'Q' untuk keluar", (20, 90), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 255, 0), 2)

    cv2.imshow("Absensi", frame)
    key = cv2.waitKey(1)

    if key == ord('q'):
        break
    elif key == ord('a'):
        addNewUser()

videoCapture.release()
cv2.destroyAllWindows()

Pengguna Afrino dengan nomor registrasi 001 berhasil ditambahkan.
